In [2]:
from numpy import linalg as LA
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import timeit
import time
import os
%run ../functions.ipynb

## 影像處理

In [3]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return blur, nlm, otsu, edge

def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## Get subpixel edge locations

In [4]:
def sigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))

In [5]:
def get_influential_neighbors(coord):
    ## 依照 sample 的點落在當下 pixel 中的哪個空間，分為 4 個case
    ##     0 . 1
    ##    .......
    ##     2 . 3
    x = round(coord[0])
    y = round(coord[1])
    a = y > coord[0] and 1 or 0
    b = x > coord[1] and 1 or 0
    space = np.dot([1,2], [a,b])
    
    switcher = {
        0: [[y,x], [y-1,x], [y-1,x-1], [y,x-1]],
        1: [[y,x], [y,x+1], [y-1,x+1], [y-1,x]],
        2: [[y,x], [y,x-1], [y+1,x-1], [y+1,x]],
        3: [[y,x], [y+1,x], [y+1,x+1], [y,x+1]],
    }
    
    neighbors_loc = np.asarray(switcher.get(space)).astype(int)
    return neighbors_loc

In [6]:
def get_negihbors_val(neighbors_loc, img):
    neighbors_val = [img[y,x] for y, x in neighbors_loc]
    return neighbors_val

In [7]:
def get_negihbors_wt(coord, neighbors_loc): 
    ## 取的每個 neighbor 與 sample 點的距離
    neighbors_dist = LA.norm(coord - neighbors_loc, axis=1)
    neighbors_dist = neighbors_dist / np.sum(neighbors_dist)

    ## 若與第一個 pixel 的距離是 0 (即在 pixel 上)則為當下的值
    ## 否則每個 neighbor 的權重為距離倒數
    neighbors_wt = neighbors_dist[0] == 0 and [1,0,0,0] or (1 - neighbors_dist)/3
    return neighbors_wt

In [8]:
def get_spl_val(coord, img):
    ## 取得有影響力的 4 個相鄰點及相對應的 值 和 權重
    neighbors_loc = get_influential_neighbors(coord)
    neighbors_val = get_negihbors_val(neighbors_loc, img)
    neighbors_wt  = get_negihbors_wt(coord, neighbors_loc)
    
    ## 計算 sample 的值
    value = np.dot(neighbors_val, neighbors_wt)  
    return value
    

In [9]:
def get_sbp_edges(center, edge_pts, img):

    sbp_edges = []
    for pt in edge_pts:
        ## 取得中心到邊緣的單位向量
        c2e_V = pt - center
        c2e_V = c2e_V/LA.norm(c2e_V)
        
        ## 取得 sapmple 點的'座標'及'亮度值' 
        spl_pts  = [pt + i*c2e_V for i in range(-5,6)]       # 需要再找道合適參數
        spl_vals = [get_spl_val(spl_pt, img) for spl_pt in spl_pts]

        ## 擬合 S 曲線
        x_label = np.asarray(range(0,11))
        y_vals  = (spl_vals - min(spl_vals))/ (max(spl_vals) - min(spl_vals))
        popt, pcov = curve_fit(sigmoid, x_label, y_vals, maxfev=10000)
        
        
        ## Debug
#         print(popt[1])
#         x = np.linspace(0, 11)
#         y = sigmoid(x, *popt)

#         plt.plot(x_label, y_vals, 'o', label='data')
#         plt.plot(x,y, label='fit')
#         plt.ylim(0, 1)
#         plt.legend(loc='best')
#         plt.show()
        
        
        ## 取得 Subpixels' 的位置
        sbp = pt - 1*(5-popt[1])*c2e_V
        sbp_edges.append(sbp)

        
    return sbp_edges

In [9]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
# os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file = "10_mean_1.bmp"
img = cv2.imread(file, 0)


blur, nlm, otsu, edge = image_processing(img)
edge_points = get_edge_points(edge)
cx,cy,r,_ = cf.least_squares_circle((edge_points))
sp_edges = get_sbp_edges(np.asarray([cx,cy]), edge_points, img)

# cx,cy,r,_ = cf.least_squares_circle((sp_edges))
# print("{}, {}".format(cx, cy))

# x = [a[0] for a in edge_points]
# y = [a[1] for a in edge_points]
# plt.plot(x, y, 'o', label='ori', markersize=1)
# x = [a[0] for a in sp_edges]
# y = [a[1] for a in sp_edges]
# plt.plot(x, y, 'o', label='sub', markersize=1)
# plt.legend(loc='best')
# plt.axis('equal')
# plt.show()

# cv2.imshow("Image", nlm)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
try:
    images = os.listdir()
    images.sort()
    
    predictions = []
    for image in images:
        img = cv2.imread(image, 0)

        blur, nlm, otsu, edge = image_processing(img)
        edge_points = get_edge_points(edge)
        cx,cy,r,_ = cf.least_squares_circle((edge_points))
        sp_edges = get_sbp_edges(np.asarray([cx,cy]), edge_points, nlm)

        cx,cy,r,_ = cf.least_squares_circle((sp_edges))
        predictions.append([cx, cy])
        print("{}: {}, {}".format(image, cx, cy))

except Exception as e:
    print(e) 

10_mean_0.bmp: 203.92374210347566, 99.06393226476749
10_mean_1.bmp: 203.95786046389995, 99.0298274387758
10_mean_10.bmp: 203.9256374033678, 98.8176735595217
10_mean_11.bmp: 203.93448766930086, 98.91706417540894
10_mean_12.bmp: 203.8140103845119, 98.97741490167668
10_mean_13.bmp: 203.942244316097, 98.86883392157728
10_mean_14.bmp: 203.79314491628185, 99.01153909104941
10_mean_15.bmp: 203.772963546671, 98.92249539246583
10_mean_16.bmp: 203.8732492044871, 98.98213863237254
10_mean_17.bmp: 203.94413278602178, 99.02150672652718
10_mean_18.bmp: 203.80747082546162, 98.8366205255092
10_mean_19.bmp: 203.92186053395082, 98.92268100618087
10_mean_2.bmp: 203.76127448766806, 98.78027689271794
10_mean_20.bmp: 203.83257122698237, 98.86221255568061
10_mean_21.bmp: 203.82758777193874, 98.99908110555668
10_mean_22.bmp: 203.86173224545274, 98.91746734870091
10_mean_23.bmp: 203.82640007765028, 98.94789345343169
10_mean_24.bmp: 204.01091668410305, 98.85757858430176
10_mean_25.bmp: 204.05383443303646, 98.91

In [11]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (203.90399, 98.94192)
Variance     : (0.00919, 0.01383)
Max Length   : 0.56520, 0.20093 (pixels)
